In [ ]:
# setting up jupyter
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['savefig.dpi'] = 300

# Example 1: successive differentiation of hESCs to heptocytes
In this experiment human embryonic stem cells were treated with a selection of stimuli to differentiate into hepatocytes/ liver tissue. Within the a period of 3 weeks, 10 samples were taken and sequenced on different days. This example goes into samples and targets initiation and basic plotting.

## Initilizing samples and targets
To <b>initiate the samples instance</b> we pass a TSV expression file and gene list data. Since there are no replicates in this data, we cannot identify significant differential genes with DEseq2 (of which DPre accepts the output). Instead, in this analysis, differential genes were defined as those with an absolute Z-score >2. The genes exceeding this threshold were saved in the two directories <i>DPre/examples/hsliver/up_genelists</i> and <i>DPre/examples/hsliver/down_geneslists</i>.
Additionally, we pass the name of the control to the <i>ctrl</i> argument to later plot differential effects. We also set a <i>name</i> that will appear in logs and plot annotations.

In [ ]:
from DPre import samples
s = samples(expression = 'hsliver_expression.tsv',     # expression input data
            diff_genes = ('./up_genelists', './down_genelists'),    # differential gene list directories
            ctrl = 'Day00',     # name of the control in the data
            name = 'in vitro hepatic differentiated hESCs'    # name of the samples instance
)

No alignment issues were detected between the two inputs. Now we have initiated a fully functional samples instance that can be used for rating transcriptional similarity.
 

The <b>targets initiation</b> is easy because we want to compare against the pre-packaged human reference data. This can simply be realized by calling the <i>preset_targets()</i> function with 'human'.

In [ ]:
from DPre import preset_targets
t = preset_targets('human')     # initiate one of the pre-packaged targets, here the human reference

Because we did not set the <i>preset_colors</i> argument to <i>None</i>, targets default colors were set according to the respective domain. In addition, a color legend is created to reference the mapping of domains and colors. 

The <b>command line interface</b> combines targets and samples initiation with producing the plot. Without the plot defined, the command is invalid. But for demonstration purposes the equivalent would be:

In [ ]:
!python ../../dpre.py --preset_targets "human" --samples_expression "hsliver_expression.tsv" --samples_diff_genes "./up_genelists" --samples_diff_genes "./down_genelists" --control "Day00"  --samples_name "in vitro hepatic differentiated hESCs"

or with abbriviated arguments:

In [ ]:
!python ../../dpre.py -pt "human" -se "hsliver_expression.tsv" -sd "./up_genelists" -sd "./down_genelists" -c "Day00"  -sn "in vitro hepatic differentiated hESCs"

Now that both the targets and samples are defined, we can <b>compute and plot the transcriptional similarity</b> between the two.

## Plotting  target transcriptional similarity

First, we want to get an initial overview of the data. To produce this overview, the <b>target similarity heatmap</b> is called from the targets instance. Here we first want to check the <b>'euclid' metric</b>. 
Note: in the following plot calls we pass the argument <i>plt_show = True</i>. This is simply for displaying the plot in jupyter and will default to <i>False</i>. Because we want to pivot the plot later, we will first reverse the samples order like so:

In [ ]:
rev_order = list(reversed(s.names))
s.reorder(rev_order)

In [ ]:
hm = t.target_similarity_heatmap(samples = s,     # the samples to check similarity for
                                 which = 'euclid',     # the metric to use for computing similarity
                                 filename = None,     # don't save this initial plot
                                 plt_show = True,     # show the plot in jupyter notebooks
                                )

The log shows normal proportions of detected marker genes in the samples. No targets are dropped for the analysis.
As we can see, the default paramters of the target similarity heatmap are set to show the target labels. The plot is created with the space necessary to cleanly display all target labels. Therefore, when comparing against a large reference the plot widely extends in x-direction. Although this can be useful for reading out the similarities of all targets, we cannot present it in this size. To solve this, we pass <i><b>heatmap_width</b></i>, <b><i>hide_targetlabels</i></b>, and <b><i>targetlabels_space</i></b>

In [ ]:
hm = t.target_similarity_heatmap(samples = s,
                                 which = 'euclid',    
                                 heatmap_width = .08, 
                                 hide_targetlabels = True,     # target labels are not displayed
                                 targetlabels_space = .1,     # size in inches under the heatmap
                                 filename = None,
                                 plt_show = True,     # show the plot in jupyter notebooks
                                )

To further reduce the plot to the essentials, we pass <b><i>hide_targets_colorbar</i></b>, <b><i>hide_distance_bar</i></b>, <b><i>samplelabels_space</i></b> and set <b><i>title</i></b> to <i>None</i>. Also, we pass <b><i>pivot</i></b> to flip the plot 90 degrees and <b>filename</b>. This is why we reordered the samples in the beginning.

In [ ]:
hm = t.target_similarity_heatmap(samples = s, 
                                 which = 'euclid',     # the metric to use for computing similarity
                                 heatmap_width = .08,
                                 hide_targetlabels = True,
                                 targetlabels_space = .1,
                                 hide_targets_colorbar = False,      # show the colorbar indicating target domains
                                 hide_distance_bar = True,      # hide the distance bar, for intersect n marker genes
                                 samplelabels_space = .3,
                                 title = None,
                                 pivot = True,
                                 filename = 'target_sim_euclid.png',     # filename for saving
                                 plt_show = True,
                                )

The same output can be achieved by calling the following <b>command</b>:

In [ ]:
!python ../../dpre.py -pt "human" -se "hsliver_expression.tsv" -sd "./up_genelists" -sd "./down_genelists" -c "Day00"  -sn "in vitro hepatic differentiated hESCs" target_sim -w "euclid" -hw 0.08 -hta -ta 0.1 -htc -hd -sa 0.3 -t "most similarity increasing genes" -pi -f "target_sim_euclid.png" 

The intention of this example is to showcase and compare the two metrics 'euclid' and 'intersect'. Let's run the same plot with the <b>'intersect' metric</b>:

In [ ]:
hm = t.target_similarity_heatmap(samples = s, 
                                 which = 'intersect',     # the metric to use for computing similarity
                                 heatmap_width = .08,
                                 hide_targetlabels = True,
                                 targetlabels_space=.1,
                                 hide_targets_colorbar = True,      # hide the colorbar indicating target domains
                                 hide_distance_bar = False,      # show the distance bar, for intersect n marker genes
                                 samplelabels_space = .3,
                                 title = None,
                                 pivot = True,
                                 filename = None,
                                 plt_show = True,
                                )

A first comparison between the two metrics seems to indicate notable differences. However, this is only due to a common problem in weighting the similarity scores. When <i><b>proportional</b></i> is <i>False</i>, the raw number of matches with the target marker genes is shown. This has the obvious effect that targets with high number of marker genes will score higher. We can deal with this problem by setting <i>proportional = True</i> to devide the score by the number of marker genes:

In [ ]:
hm = t.target_similarity_heatmap(samples = s, 
                                 which = 'intersect',
                                 proportional = True,     # devide score by the number of markeregenes
                                 heatmap_width = .08,
                                 hide_targetlabels = True,
                                 targetlabels_space=.1,
                                 hide_targets_colorbar = True,
                                 hide_distance_bar = False,
                                 samplelabels_space = .3,
                                 title = None,
                                 pivot = True,
                                 filename = 'target_sim_intersect.png',
                                 plt_show = True,
                                )

This matches the results of the 'euclid' metric much better. In both heatmaps we can make out 3 distinct targets that increase in similarity, namely Liver (adult), Liver (fetal) and Hepatocyte.

## Plotting single gene transcriptional similarity

Next we want to investigate the transcriptional similarity with the hepatocyte target for two groups of <b>genes</b>. First, known hepatocyte marker genes and next the genes driving the similarity increase most dominently. This functionality is provided by the <i>gene_similarity_heatmap()</i> function. The function will create a single gene similarity plot for every element in the targets instance. Since we are only interested in Hepatocyte similarity, we can <b>slice the elements</b> in targets like so:

In [ ]:
hep = t.slice_elements(['Hepatocyte'], name='Hepatocyte')

In [ ]:
hm = hep.gene_similarity_heatmap(samples = s,
                                 filename = None,
                                 plt_show = True)

By default, this function will plot the 45 most variant genes, we are however intersted in the <b>similarity of a specifc gene list</b>: a collection of hepatoblast- and hepatocyte markers. We define these genelists and generate a color legend for the two groups. Then we make a dictionary that maps the genes to the respective color. DPre's predefined <b>colors</b> can be accessed from <i>config.colors</i>:

In [ ]:
from DPre import plot_color_legend
from DPre import config
hep_bl = ['PITX2', 'KRT19', 'AFP', 'DAB2', 'FOXP4', 'SMAD3', 'FOXA1', 'HES4', 'HNF1B']
hep_cy = ['ALB', 'MXD1', 'TTR']
plot_color_legend(['heptoblast', 'heptocytes'], 
                  [config.colors[11], config.colors[12]],
                  filename='hepato_markers_legend.png'
                 )
# set up the colors for the colorbar
colorbar_cols = {**dict.fromkeys(hep_bl, config.colors[11]), 
                 **dict.fromkeys(hep_cy, config.colors[12])}
colorbar_cols

To check the similarity of a custom gene list, this list is passed to <i><b>custom_target_genelist</b></i>. Since we will plot another gene similarity heatmap, we want to control the <i><b>heatmap_range</b></i> instead of inferring it from the data. Due to space constrains, the summary plot on the right is not plotted. Apart from these arguments, the other inputs overlap with the previous <i>target_similarity_heatmap()</i> function:

In [ ]:
hm = hep.gene_similarity_heatmap(samples = s,    # the sampels to explore similarity for
                                 custom_target_genelist = hep_bl+hep_cy,    # the list of genes to be visualized
                                 heatmap_range = [-6, 6],     # range heatmap values
                                 pivot = True,
                                 heatmap_width = .7, 
                                 heatmap_height = .9, 
                                 genelabels_space = .3, 
                                 samplelabels_space_left = .4, 
                                 title = 'hepatoblast & heptocytes markergenes',
                                 show_genes_colorbar = colorbar_cols,    # the colorbar dictionary created above
                                 hide_distance_bar = True,
                                 hide_sum_plot = True,     # do not plot the summerizing plot on the right
                                 filename = 'hsliver_marker_genes.png',
                                 plt_show = True,
                                )


The <b>command line interface</b> doesn't support custom colors. Apart from this the plot can be reproduced like so:

In [ ]:
!python ../../dpre.py -pt "human" -ts "Hepatocyte" -se "./hsliver_expression.tsv" -sd "./up_genelists" -sd "./down_genelists" -c "Day00"  -sn "in vitro hepatic differentiated hESCs" gene_sim -cu "PITX2" -cu "KRT19" -cu "AFP" -cu "DAB2" -cu "FOXP4" -cu "SMAD3" -cu "FOXA1" -cu "HES4" -cu "HNF1B" -cu "ALB" -cu "MXD1" -cu "TTR" -hr -6 -hr 6 -pi -hw 0.7 -hh 0.9 -ge 0.3 -sa 0.4 -t "hepatoblast & heptocytes markergenes" -hd -hs -f "hsliver_marker_genes.png"

Second, we want to find the 10 most similarity-increasing genes for the haptocyte fate. Instead of passing <i>custom_target_genelist</i>, we change <b><i>display_genes</i></b> and <b><i>gene_number</i></b>. Here, we do want to show the summary plot but the size should be reduced slightly. Advanced size adjustments like this are realized by passing the size constants defined in <i>config</i> (see documentation for details), in this case <i>G_HM_SUMPLOT_SIZE</i>. This functionality is not available through the command line.

In [ ]:
                             
hm = hep.gene_similarity_heatmap(samples = s, 
                                 display_genes = 'increasing',      # the genes with peak positve similarity values
                                 gene_number = 10,        # the number of genes to extract and display
                                 plt_show = True,
                                 heatmap_range = [-6, 6],
                                 pivot = True,
                                 heatmap_width = .7, 
                                 heatmap_height = .9, 
                                 genelabels_space = .3, 
                                 samplelabels_space_left = .4, 
                                 title = 'most similarity increasing genes',
                                 hide_distance_bar = True,
                                 hide_samplelabels = True,     # the label with be shared with the marker genes plot above
                                 hide_sum_plot = False,     # show the summary plot
                                 G_HM_SUMPLOT_SIZE = .65,        # edit the size constant defined in the config module
                                 filename = 'hsliver_increasing_genes.png',
                                )

which is equivalent (without adjusting the sum plot size) to the <b>command</b>:

In [ ]:
!python ../../dpre.py -pt "human" -ts "Hepatocyte" -se "hsliver_expression.tsv" -sd "./up_genelists" -sd "./down_genelists" -c "Day00"  -sn "in vitro hepatic differentiated hESCs" gene_sim -di "increasing" -gn 10 -hr -6 -hr 6 -pi -hw 0.7 -hh 0.9 -ge 0.3 -sa 0.4 -t "most similarity increasing genes" -hd -hsa -f "hsliver_increasing_genes.png"

Done. We now have an idea about the genes behind the differentiation bias towards hepatocytes. 